In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil
import os

from skimage.color import rgb2lab, lab2rgb
from skimage.io import imread, imshow

In [ ]:
TRAIN_ROOT_DIR = "/images/training_set" #route for training set of the dataset
DEST = "/train/images" #image folder of the dataset

if not os.path.exists(DEST):
  os.makedirs(DEST)

  for dir in os.listdir(TRAIN_ROOT_DIR):
    for img in os.listdir(os.path.join(TRAIN_ROOT_DIR, dir)):
      ORG = os.path.join(TRAIN_ROOT_DIR, dir, img)

      try:
        imread(ORG)
        shutil.copy(ORG, DEST)
      except:
        print(f"the image at {ORG} cannot be used")

In [ ]:
len(os.listdir("/train/images"))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(featurewise_center=True, 
                                   rotation_range=40, 
                                   horizontal_flip=True, 
                                   vertical_flip = True,
                                   rescale = 1/255)

In [ ]:
train = train_datagen.flow_from_directory("dataset_updated\\train", target_size = (256, 256), batch_size= 1500, shuffle=True)

# Visualization

In [ ]:
t_img , label = train.next()

def plotimage(img_arr, label):
    """
    input:image array
    output:plots the image
    """
    plt.figure(figsize=(5, 5))
    for im, l in zip(img_arr, label):
        plt.imshow(im)
        plt.title(im.shape)
        plt.axis('off')
        plt.show()

In [ ]:
x = []
y = []

In [ ]:
from skimage.color import rgb2lab

for image in t_img:
    try:
        lab = rgb2lab(image)
        x.append(lab[:, :, 0])
        y.append(lab[:, :, 1:] / 128)
    except Exception as e:
        print('Error:', e)

In [ ]:
x_train = np.array(x)
x_train = np.expand_dims(x_train, axis =len(x_train.shape))

y_train = np.array(y)
x_train.shape, y_train.shape

# Build Encoder Decoder Model

In [ ]:
from keras.layers import Conv2D, MaxPool2D, UpSampling2D, Input, LeakyReLU
from keras.models import Sequential

model = Sequential()
model.add(Conv2D(64,(3,3), activation='relu', padding='same', strides=2, input_shape=(256, 256, 1)))
model.add(Conv2D(128,(3, 3), activation='relu', padding='same'))
model.add(Conv2D(128,(3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256,(3, 3), activation='relu', padding='same'))
model.add(Conv2D(256,(3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512,(3, 3), activation='relu', padding='same'))
model.add(Conv2D(512,(3, 3), activation='relu', padding='same'))
model.add(Conv2D(256,(3, 3), activation='relu', padding='same'))

model.add(Conv2D(128,(3, 3), activation='relu', padding= 'same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64,(3, 3), activation='relu', padding= 'same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32,(3, 3), activation='relu', padding= 'same'))
model.add(Conv2D(16,(3, 3), activation='relu', padding= 'same'))
model.add(Conv2D(2,(3, 3), activation= LeakyReLU(alpha=0.01), padding = 'same'))
model.add(UpSampling2D((2,2)))

model.summary()

# Compile the Model

In [ ]:
model.compile(optimizer='adam', metrics = ['acc'], loss = 'mse')

In [ ]:
Model = model.fit(x_train, y_train, epochs =50, batch_size = 64, steps_per_epoch=x_train.shape[0]//64, verbose =1)

# Save Model

In [ ]:
model.save("./ImageColorizer.h5")

In [ ]:
from keras.preprocessing.image import img_to_array, load_img
img1 = []
path = ""
img = img_to_array(
    load_img(path, target_size=(256, 256, 3))
)/255

plt.title("input img")
imshow(img)
plt.axis("off")
plt.show()

In [ ]:
img1.append(img)
img1 = rgb2lab(img1)[:,:,:,0]
img1 = img1.reshape(img1.shape+(1,))

In [ ]:
output = model.predict(img1)
output = output*128

result = np.zeros((256, 256, 3))
result[:,:,0] = img1[0][:,:,0]
result[:,:,1:] = output[0]

imshow(lab2rgb(result))